In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib import style
style.use('ggplot')

### Motivation - 
##### Why risk management is to crucial in banking business and how banking risk management is different from other businesses ?
##### Leverage = Capital/Total Assets. As per the Bassel 2 norms leverage should be at least 4%.
##### If bank's losses excceds 4% of the total assets, that is, if the bank in not able to recover at least 96% of the loans or asset value losses more than 4%, then the bank will be burst. Looking that the bank's role in the economy, banks should be protected. Hence robust risk management is required.  

### Risk management workflow
#### Identify > Quantify > Mitigate

##### Types of risks associated with the banking business - Market risk, Credit risk, Liquidity risk, Solvency risk etc.
##### We use different analytical tools to quantify the different risks
##### We will discuss market risk in this notebook 

### Data Setup and EDA

In [ ]:
index_data=pd.read_csv('../input/index-data/Sensex.csv',)
index_data.head()

In [ ]:
index_data.tail()

In [ ]:
index_data['Close'].tail()

In [ ]:
fig,ax=plt.subplots(figsize=(12,4))

ax.plot(index_data['Date'],index_data['Close'])
ax.set(xlabel='Date',ylabel='Sensex Closing',title='Daily Index Movement')
ax.xaxis.set_major_locator(mdates.YearLocator())

In [ ]:
index_returns=index_data['Close'].pct_change()
index_returns.head()

In [ ]:
type(index_returns)

In [ ]:
nornal_returns=(np.random.normal(0,index_returns.dropna().std(),1294))

In [ ]:
fig,ax=plt.subplots()
sns.kdeplot(index_returns.dropna(),ax=ax);
sns.kdeplot(nornal_returns,color='b',ax=ax);

#### The first look at the index return shows that the daily Sensex returns are approximately normally distributed however there are outliers
#### For VaR calculation, the assumption is that the returns are normally distributed. Is it safe to use normal distribution to approximate stock or market index returns ?
#### Distribution of daily index returns are rather symmetric about it's mean, but the tails are fatter, that is more outliers, than what would be expected with the normal distribution.

In [ ]:
sns.displot(index_returns.dropna(),kind='kde');

In [ ]:
index_returns.plot(kind='box');

#### The box plot highlights the fact that there are significant outliers

In [ ]:
outlier_data=index_data[(index_returns<-0.05) | (index_returns>0.05)]
outlier_data

In [ ]:
outlier_data=outlier_data.reset_index()

In [ ]:
outlier_data

In [ ]:
outlier_data['Previous Close']=0
outlier_data

In [ ]:
for i in range(len(outlier_data)):
    outlier_data.iloc[i,6]=index_data.iloc[outlier_data['index'][i]-1]['Close']

In [ ]:
outlier_data

##### Outlier data can be further investigated for the reason for the daily returns more than +/- 5%

### Simulation

In [ ]:
last_price=index_data['Close'].iloc[-1]

In [ ]:
last_price

In [ ]:
num_simulations=1000
num_days=252

In [ ]:
simulated_data=pd.DataFrame()

In [ ]:
for x in range(num_simulations):
    count=0
    daily_volatility=index_returns.std()
    
    price_series=[]
    
    price=last_price*(1+np.random.normal(0,daily_volatility))
    price_series.append(price)
    
    for y in range(num_days):
        if count==251:
            break
        price=price_series[count]*(1+np.random.normal(0,daily_volatility))
        price_series.append(price)
        count+=1
    
    simulated_data[x]=price_series

In [ ]:
fig=plt.figure()
fig.suptitle('Monte Carlo Simulation Sensex')
plt.plot(simulated_data)
plt.axhline(y=last_price,color='r',linestyle='-')
plt.xlabel('Day')
plt.ylabel('Index level')

In [ ]:
fig, ax = plt.subplots()
for i in range(50):
    sns.kdeplot(simulated_data[i].pct_change().dropna(),ax=ax)

#### distribution of returns of the first 50 simulated samples

### VaR - Value At Risk

##### VaR = position X stand dev of stock or index returns X z score for 0.95

In [ ]:
import scipy
alpha=scipy.stats.norm.ppf(0.95)
alpha

##### We wanted to know how much the market could possibly move against our position. VaR can be interpreated as - we are 95% confident that the daily loss would not exceed the VaR amount, calculated as below 

In [ ]:
position=100000
var=position*index_returns.dropna().std()*alpha
var

#### VaR is the additional capital we would need to support the position. In our example, to keep the long position of 100,000 we need capital of 1912

### Stress Testing

#### In simulation section we took stand. dev. of daily closing prices are measure of volatality to simulate the future prices. However what if the daily volatality increases ? How much the additinal capital the bank need to support its positions ?

In [ ]:
daily_volatility=index_returns.std()
print(f'Daily Volatality of the markt index returns is {daily_volatility} %')
print(f'Daily Volatality of the markt index returns is {(((1+daily_volatility)**12)-1)*100} %')

##### If daily volatality increase by 5%, 10%; what would be the VaR ? That is how much additinal capital the bank would need to support its position ?

In [ ]:
# If the market index return volatality increased to 20%
daily_vol_20=((0.20+1)**(1/12))-1
print(f'For yearly volatility of the market index returns of 20%,\nthe daily volatility of the index returns is {daily_vol_20}')
var_20=position*daily_vol_20*alpha
print(f'Daily VaR at yearly volatality of 20% is {var_20}')

In [ ]:
# If the market index return volatality increased to 20%
daily_vol_25=((0.25+1)**(1/12))-1
print(f'For yearly volatility of the market index returns of 25%,\nthe daily volatility of the index returns is {daily_vol_25}')
var_25=position*daily_vol_25*alpha
print(f'Daily VaR at yearly volatality of 20% is {var_25}')

### VaR Simulation

In [ ]:
simulated_volatality=pd.DataFrame(columns=['simulated_volatality','simulated_var'])

In [ ]:
for x in range(num_simulations):
    simulated_volatality.loc[x,'simulated_volatality']=simulated_data.iloc[:,x].pct_change().std()
    simulated_volatality.loc[x,'simulated_var']=position*simulated_volatality.loc[x,'simulated_volatality']*alpha

In [ ]:
simulated_volatality.head()

In [ ]:
plt.plot(simulated_volatality['simulated_var']);

In [ ]:
mean_simulated_var=np.mean(simulated_volatality['simulated_var'])
stand_dev_simulated_var=np.std(simulated_volatality['simulated_var'])
print(f'Simulated average daily VaR is {mean_simulated_var} with standard deviation of {stand_dev_simulated_var}')

### Conclusion : The techniques of simulations and synthesizing data are invaluable in generating and testing hypothesis about the data. In this case we simulated the market index data and tried to find how much capital a bank should keep aside to fund its position. This can be utilized for optimum capital allocation.  